<a href="https://colab.research.google.com/github/RezSat/synthetic-data-generator/blob/main/PaddleOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!cp -r /content/drive/MyDrive/runs /content/

In [6]:
!cp -r /content/drive/MyDrive/yolo_dataset /content/

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.3 MB/s eta 0:00:00


In [ ]:
# 1. Update package list
!sudo apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,289 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,637 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,599 kB]
Get

In [ ]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())


None
False


In [1]:
# 1. Uninstall the breaking versions
!pip uninstall -y langchain langchain-community langchain-core paddlex

# 2. Install the specific compatible version (Pre-v0.1.0)
# This version still has 'langchain.docstore' where Paddle expects it.
!pip install langchain==0.0.354

# 3. Re-install PaddleOCR essentials
!pip install paddlepaddle-gpu==2.6.1
!pip install "paddleocr>=2.7.0" ultralytics opencv-python-headless

print("✅ Fix applied. PLEASE RESTART THE RUNTIME (Runtime -> Restart Session).")

Found existing installation: langchain 0.0.354
Uninstalling langchain-0.0.354:
  Successfully uninstalled langchain-0.0.354
Found existing installation: langchain-community 0.0.20
Uninstalling langchain-community-0.0.20:
  Successfully uninstalled langchain-community-0.0.20
Found existing installation: langchain-core 0.1.23
Uninstalling langchain-core-0.1.23:
  Successfully uninstalled langchain-core-0.1.23
  Using cached langchain-0.0.354-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.0.37-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.36-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_community-0.0.35-py3-none-any.whl.meta

✅ Fix applied. PLEASE RESTART THE RUNTIME (Runtime -> Restart Session).


In [4]:
!pip install -U requests

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [1]:
import os
import cv2
import json
import logging
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
import matplotlib.pyplot as plt

# ==========================================
# 1. CONFIGURATION & INITIALIZATION
# ==========================================
os.environ["DISABLE_MODEL_SOURCE_CHECK"] = "True"
# Suppress Paddle's verbose logging
os.environ["GLOG_minloglevel"] = "3"
logging.getLogger("ppocr").setLevel(logging.ERROR)

# Initialize PaddleOCR with GPU support
# lang='sin' handles Sinhala. For mixed Tamil, you might need a secondary pass or lang='ta'
print("🚀 Initializing PaddleOCR Engine (GPU)...")
ocr_engine = PaddleOCR(
    use_angle_cls=True,
    lang='sin',            # Your requested config
    ocr_version='PP-OCRv3',
    use_gpu=True,
    show_log=False,        # Keep output clean
    det=False,             # IMPORTANT: We disable Paddle's detection. We trust YOLO.
    rec=True,              # We only use Recognition
    cls=True,               # Angle classification helps if scans are skewed
    use_mp=False
)
print("✅ PaddleOCR Initialized.")

# ==========================================
# 2. HELPER FUNCTIONS
# ==========================================

def surgical_crop(image, box):
    """
    Crops the image based on YOLO box coordinates with boundary checks.
    box: [x1, y1, x2, y2]
    """
    h, w, _ = image.shape
    x1, y1, x2, y2 = map(int, box)

    # Padding (optional): Add 2-3 pixels to ensure characters on edge aren't cut
    pad = 2
    x1 = max(0, x1 - pad)
    y1 = max(0, y1 - pad)
    x2 = min(w, x2 + pad)
    y2 = min(h, y2 + pad)

    return image[y1:y2, x1:x2]

def run_paddle_extraction(image_path, yolo_model_path):
    """
    Full Pipeline:
    1. YOLO Detects Layout
    2. Crops Regions
    3. PaddleOCR Extracts Text per Region
    4. Returns Master JSON
    """
    print(f"📄 Processing: {image_path}")

    # 1. Load Image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found at {image_path}")

    # 2. Load YOLO Model
    print(f"🧠 Loading YOLO model: {yolo_model_path}")
    model = YOLO(yolo_model_path)

    # 3. Run Inference (Layout Analysis)
    results = model.predict(source=img, conf=0.5, save=False) # conf=0.5 is a safe baseline
    result = results[0] # Get first image result

    master_data = []

    # 4. Iterate through detections (The "Surgery")
    print(f"⚡ Detected {len(result.boxes)} layout elements. Starting extraction...")

    for i, box in enumerate(result.boxes):
        # Extract YOLO Data
        bbox = box.xyxy[0].cpu().numpy().tolist() # [x1, y1, x2, y2]
        cls_id = int(box.cls[0])
        cls_name = model.names[cls_id]
        conf = float(box.conf[0])

        # Skip specific classes if they don't contain text (e.g., purely visual elements)
        # Assuming all your classes (header, text_block, key_value) have text.
        # If 'checkbox_group' needs special handling (OMR), skip OCR here or handle differently.

        # 5. Crop the ROI
        roi = surgical_crop(img, bbox)

        # 6. Run PaddleOCR on the ROI
        # cls=True checks for rotation inside the crop
        ocr_result = ocr_engine.ocr(roi, cls=True)

        extracted_text = ""

        # PaddleOCR output format is complex: [[[[x,y]..], (text, conf)], ...]
        # Since we ran on a crop, we might get multiple lines. We join them.
        if ocr_result and ocr_result[0]:
            lines = [line[1][0] for line in ocr_result[0]]
            extracted_text = " ".join(lines)

        # 7. Construct JSON Object
        element_data = {
            "id": i,
            "type": cls_name,
            "bbox": [round(x) for x in bbox], # Pixel perfect coordinates
            "text": extracted_text,
            "confidence_layout": round(conf, 4)
        }

        # Log to console for sanity check
        print(f"   [{cls_name.upper()}] -> '{extracted_text}'")

        master_data.append(element_data)

    return master_data

# ==========================================
# 3. EXECUTION BLOCK
# ==========================================

# Define your paths here
# Upload your image and model to Colab content folder
YOLO_PATH = "runs/detect/sinhala_doc_layout/weights/best.pt"  # REPLACE with your trained YOLOv8 path
IMAGE_PATH = "yolo_dataset/train/complex_sample_0741.jpg" # REPLACE with your sample image

# Only run if files exist (prevents error if you just copy-paste without uploading)
if os.path.exists(YOLO_PATH) and os.path.exists(IMAGE_PATH):
    final_json = run_paddle_extraction(IMAGE_PATH, YOLO_PATH)

    # Save Output
    output_filename = "reconstruction_data.json"
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(final_json, f, ensure_ascii=False, indent=4)

    print(f"\n✅ Extraction Complete. Data saved to {output_filename}")

    # Optional: Preview first few characters of JSON
    print(json.dumps(final_json[:2], indent=2, ensure_ascii=False))
else:
    print("⚠️ Please upload your 'best.pt' and a 'form_sample.jpg' to the Colab files area.")

Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.


🚀 Initializing PaddleOCR Engine (GPU)...


ValueError: No models are available for the language 'sin' and OCR version 'PP-OCRv3'.

#  Training PaddleOCR

In [ ]:
import os
import random

# ====================================================
# 1. SETUP DIRECTORIES & CONFIG
# ====================================================
WORK_DIR = "/content/PaddleOCR"
DATA_DIR = "/content/train_data"       # Where your images are
LABEL_FILE = "/content/all_labels.txt" # Your master labels: "img1.jpg \t Text"

# Install PaddleOCR Repo for access to training tools
if not os.path.exists(WORK_DIR):
    !git clone https://github.com/PaddlePaddle/PaddleOCR.git
    %cd {WORK_DIR}
    !pip install -r requirements.txt
    !pip install paddlepaddle-gpu
    # Install libs for faster training
    !pip install lmdb

# ====================================================
# 2. DICTIONARY GENERATION (CRITICAL)
# ====================================================
# Paddle needs a .txt file listing every unique character in your dataset.
# If this misses a character, the model will crash or ignore it.

def build_dictionary_and_split(label_file, train_ratio=0.9):
    print("Example label entry expected: 'folder/img.jpg <tab> SomeText'")

    unique_chars = set()
    train_lines = []
    val_lines = []

    # Add a space character manually (often missed)
    unique_chars.add(" ")

    with open(label_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    for line in lines:
        try:
            path, text = line.strip().split('\t', 1)
            # Collect chars
            for char in text:
                unique_chars.add(char)

            # Split dataset
            if random.random() < train_ratio:
                train_lines.append(f"{path}\t{text}")
            else:
                val_lines.append(f"{path}\t{text}")
        except ValueError:
            print(f"⚠️ Skipping malformed line: {line.strip()}")

    # Write Dictionary
    dict_path = os.path.join(WORK_DIR, "dict_sinhala.txt")
    with open(dict_path, 'w', encoding='utf-8') as f:
        for char in sorted(list(unique_chars)):
            f.write(char + '\n')

    # Write Train/Val lists
    with open(os.path.join(WORK_DIR, "train_list.txt"), 'w', encoding='utf-8') as f:
        f.write('\n'.join(train_lines))

    with open(os.path.join(WORK_DIR, "val_list.txt"), 'w', encoding='utf-8') as f:
        f.write('\n'.join(val_lines))

    print(f"✅ Dictionary built: {len(unique_chars)} unique characters.")
    print(f"✅ Data split: {len(train_lines)} Train / {len(val_lines)} Val.")
    return dict_path

# POINT THIS TO YOUR ACTUAL LABEL FILE
# If you don't have one yet, upload your 'train_data' folder and a 'labels.txt'
# build_dictionary_and_split("your_uploaded_labels.txt")